In [1]:
import pandas as pd
import numpy as np
import ops

## Test Kahan Summuation

In [2]:
small = np.full(10_000, 1e-6)
data = np.concatenate(([1e4], small))
df = pd.DataFrame({"x": data})
df["y"] = df["x"].rolling(len(df), min_periods=1).sum()
df["z"] = ops.rolling_sum(data, len(df), 1)
df

,x,y,z
0,10000.000000,10000.000000,10000.000000
1,0.000001,10000.000001,10000.000001
2,0.000001,10000.000002,10000.000002
3,0.000001,10000.000003,10000.000003
4,0.000001,10000.000004,10000.000004
...,...,...,...
9996,0.000001,10000.009996,10000.009996
9997,0.000001,10000.009997,10000.009997
9998,0.000001,10000.009998,10000.009998
9999,0.000001,10000.009999,10000.009999


In [3]:
print("y == z:", df["y"].equals(df["z"]))
print("y[-1] =", df["y"][-1:].to_list()[0])
print("sum(x) =", df["x"].sum())
print("sum(x) - y[-1] =", df["x"].sum() - df["y"][-1:].to_list()[0])

y == z: True
y[-1] = 10000.01
sum(x) = 10000.010000000004
sum(x) - y[-1] = 3.637978807091713e-12


## Test Time Series Rolling Sum/Mean

In [4]:
import duckdb

ORDER_TYPE_MARKET = 1
ORDER_TYPE_LIMIT = 2
ORDER_TYPE_BEST = 3
BS_FLAG_BUY = 1
BS_FLAG_SELL = 2
MATCH_TYPE_DEAL = 1
MATCH_TYPE_DEL = 2
INNER_FLAG_OUT = 1
INNER_FLAG_IN = 2

data_dir = "../data"

con = duckdb.connect()

con.sql("""
    create or replace macro time2ms(x) as
        ((x // 10000000)::int * 3600000) +   -- hours → ms
        ((x // 100000 % 100)::int * 60000) + -- minutes → ms
        ((x // 1000 % 100)::int * 1000) +    -- seconds → ms
        (x % 1000);                          -- milliseconds        
""")

df = con.sql(f"""
    select stockid, date, seq_num, time,
        time2ms(time)::uint32 as time_ms, qty, price,
    from '{data_dir}/orders_20260119.parquet'
    where side={BS_FLAG_BUY}
        and time>=93000000 and time<145700000
""").df()
df["dt"] = pd.to_datetime(
    df["date"].astype(str) + df["time"].astype(str).str.zfill(9),
    format="%Y%m%d%H%M%S%f",
)

# add some NaNs for testing
df.loc[[1, 10, 100, 1000], "qty"] = np.nan
df

,stockid,date,seq_num,time,time_ms,qty,price,dt
0,2270,20260119,906911,93000040,34200040,1400.0,30060,2026-01-19 09:30:00.040
1,2270,20260119,907097,93000040,34200040,NaN,30060,2026-01-19 09:30:00.040
2,2270,20260119,907985,93000050,34200050,200.0,30050,2026-01-19 09:30:00.050
3,2270,20260119,908702,93000050,34200050,300.0,30020,2026-01-19 09:30:00.050
4,2270,20260119,908830,93000050,34200050,2000.0,28000,2026-01-19 09:30:00.050
...,...,...,...,...,...,...,...,...
19413,2270,20260119,57234673,145653040,53813040,800.0,32490,2026-01-19 14:56:53.040
19414,2270,20260119,57243324,145654700,53814700,1600.0,32490,2026-01-19 14:56:54.700
19415,2270,20260119,57244795,145654990,53814990,800.0,32490,2026-01-19 14:56:54.990
19416,2270,20260119,57250370,145656040,53816040,800.0,32490,2026-01-19 14:56:56.040


In [5]:
window = 30_000  # 30s
min_obs = 1

df["pd_bid_qty_sum_30s"] = df.rolling("30s", min_periods=min_obs, on="dt")["qty"].sum()
df["bid_qty_sum_30s"] = ops.ts_rolling_sum(
    df["time_ms"].to_numpy(),
    df["qty"].to_numpy(),
    window,
    min_obs,
)
df["pd_bid_qty_avg_30s"] = df.rolling("30s", min_periods=min_obs, on="dt")["qty"].mean()
df["bid_qty_avg_30s"] = ops.ts_rolling_mean(
    df["time_ms"].to_numpy(),
    df["qty"].to_numpy(),
    window,
    min_obs,
)
df

,stockid,date,seq_num,time,time_ms,qty,price,dt,pd_bid_qty_sum_30s,bid_qty_sum_30s,pd_bid_qty_avg_30s,bid_qty_avg_30s
0,2270,20260119,906911,93000040,34200040,1400.0,30060,2026-01-19 09:30:00.040,1400.0,1400.0,1400.000000,1400.000000
1,2270,20260119,907097,93000040,34200040,NaN,30060,2026-01-19 09:30:00.040,1400.0,1400.0,1400.000000,1400.000000
2,2270,20260119,907985,93000050,34200050,200.0,30050,2026-01-19 09:30:00.050,1600.0,1600.0,800.000000,800.000000
3,2270,20260119,908702,93000050,34200050,300.0,30020,2026-01-19 09:30:00.050,1900.0,1900.0,633.333333,633.333333
4,2270,20260119,908830,93000050,34200050,2000.0,28000,2026-01-19 09:30:00.050,3900.0,3900.0,975.000000,975.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
19413,2270,20260119,57234673,145653040,53813040,800.0,32490,2026-01-19 14:56:53.040,16900.0,16900.0,845.000000,845.000000
19414,2270,20260119,57243324,145654700,53814700,1600.0,32490,2026-01-19 14:56:54.700,17700.0,17700.0,885.000000,885.000000
19415,2270,20260119,57244795,145654990,53814990,800.0,32490,2026-01-19 14:56:54.990,18500.0,18500.0,880.952381,880.952381
19416,2270,20260119,57250370,145656040,53816040,800.0,32490,2026-01-19 14:56:56.040,19200.0,19200.0,914.285714,914.285714


In [6]:
df["pd_bid_qty_sum_30s"].equals(df["bid_qty_sum_30s"])

True

In [7]:
df["pd_bid_qty_avg_30s"].equals(df["bid_qty_avg_30s"])

True

## Test Time Series Rolling Min/Max

In [8]:
max_window = 30_000  # 30s
min_window = 30  # 30ms
min_obs = 1

df["pd_bid_qty_min_30ms"] = df.rolling("30ms", min_periods=min_obs, on="dt")[
    "qty"
].min()
df["bid_qty_min_30ms"] = ops.ts_rolling_min(
    df["time_ms"].to_numpy(),
    df["qty"].to_numpy(),
    min_window,
    min_obs,
)
df["pd_bid_qty_max_30s"] = df.rolling("30s", min_periods=min_obs, on="dt")["qty"].max()
df["bid_qty_max_30s"] = ops.ts_rolling_max(
    df["time_ms"].to_numpy(),
    df["qty"].to_numpy(),
    max_window,
    min_obs,
)
df

,stockid,date,seq_num,time,time_ms,qty,price,dt,pd_bid_qty_sum_30s,bid_qty_sum_30s,pd_bid_qty_avg_30s,bid_qty_avg_30s,pd_bid_qty_min_30ms,bid_qty_min_30ms,pd_bid_qty_max_30s,bid_qty_max_30s
0,2270,20260119,906911,93000040,34200040,1400.0,30060,2026-01-19 09:30:00.040,1400.0,1400.0,1400.000000,1400.000000,1400.0,1400.0,1400.0,1400.0
1,2270,20260119,907097,93000040,34200040,NaN,30060,2026-01-19 09:30:00.040,1400.0,1400.0,1400.000000,1400.000000,1400.0,1400.0,1400.0,1400.0
2,2270,20260119,907985,93000050,34200050,200.0,30050,2026-01-19 09:30:00.050,1600.0,1600.0,800.000000,800.000000,200.0,200.0,1400.0,1400.0
3,2270,20260119,908702,93000050,34200050,300.0,30020,2026-01-19 09:30:00.050,1900.0,1900.0,633.333333,633.333333,200.0,200.0,1400.0,1400.0
4,2270,20260119,908830,93000050,34200050,2000.0,28000,2026-01-19 09:30:00.050,3900.0,3900.0,975.000000,975.000000,200.0,200.0,2000.0,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19413,2270,20260119,57234673,145653040,53813040,800.0,32490,2026-01-19 14:56:53.040,16900.0,16900.0,845.000000,845.000000,800.0,800.0,2400.0,2400.0
19414,2270,20260119,57243324,145654700,53814700,1600.0,32490,2026-01-19 14:56:54.700,17700.0,17700.0,885.000000,885.000000,1600.0,1600.0,2400.0,2400.0
19415,2270,20260119,57244795,145654990,53814990,800.0,32490,2026-01-19 14:56:54.990,18500.0,18500.0,880.952381,880.952381,800.0,800.0,2400.0,2400.0
19416,2270,20260119,57250370,145656040,53816040,800.0,32490,2026-01-19 14:56:56.040,19200.0,19200.0,914.285714,914.285714,800.0,800.0,2400.0,2400.0


In [9]:
df["pd_bid_qty_min_30ms"].equals(df["bid_qty_min_30ms"])

True

In [10]:
df["pd_bid_qty_max_30s"].equals(df["bid_qty_max_30s"])

True